In [3]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import math 
import shutil 
from IPython.core import display as ICD


In [4]:
data_set=pd.read_csv("housing.csv")
print ( "Imported Successfully")

Imported Successfully


In [5]:
data_set.head(15)

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0     -122.23     37.88                41.0        880.0           129.0   
1     -122.22     37.86                21.0       7099.0          1106.0   
2     -122.24     37.85                52.0       1467.0           190.0   
3     -122.25     37.85                52.0       1274.0           235.0   
4     -122.25     37.85                52.0       1627.0           280.0   
5     -122.25     37.85                52.0        919.0           213.0   
6     -122.25     37.84                52.0       2535.0           489.0   
7     -122.25     37.84                52.0       3104.0           687.0   
8     -122.26     37.84                42.0       2555.0           665.0   
9     -122.25     37.84                52.0       3549.0           707.0   
10    -122.26     37.85                52.0       2202.0           434.0   
11    -122.26     37.85                52.0       3503.0           752.0   
12    -122.26     37.85                52.0       2491.0           474.0   
13    -122.26     37.84                52.0        696.0           191.0   
14    -122.26     37.85                52.0       2643.0           626.0   

    population  households  median_income  median_house_value ocean_proximity  
0        322.0       126.0         8.3252            452600.0        NEAR BAY  
1       2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2        496.0       177.0         7.2574            352100.0        NEAR BAY  
3        558.0       219.0         5.6431            341300.0        NEAR BAY  
4        565.0       259.0         3.8462            342200.0        NEAR BAY  
5        413.0       193.0         4.0368            269700.0        NEAR BAY  
6       1094.0       514.0         3.6591            299200.0        NEAR BAY  
7       1157.0       647.0         3.1200            241400.0        NEAR BAY  
8       1206.0       595.0         2.0804            226700.0        NEAR BAY  
9       1551.0       714.0         3.6912            261100.0        NEAR BAY  
10       910.0       402.0         3.2031            281500.0        NEAR BAY  
11      1504.0       734.0         3.2705            241800.0        NEAR BAY  
12      1098.0       468.0         3.0750            213500.0        NEAR BAY  
13       345.0       174.0         2.6736            191300.0        NEAR BAY  
14      1212.0       620.0         1.9167            159200.0        NEAR BAY

In [6]:
a=pd.DataFrame(data_set.isnull().sum())
a['# of null values'] = a[0]
b = a[['# of null values']]

In [7]:
print ( ' before Dropping Null Values : ')
print ( ' number of rows , columns : ', data_set.shape)

 before Dropping Null Values : 
 number of rows , columns :  (20640, 10)


In [8]:
ICD.display(b)

# of null values
longitude                          0
latitude                           0
housing_median_age                 0
total_rooms                        0
total_bedrooms                   207
population                         0
households                         0
median_income                      0
median_house_value                 0
ocean_proximity                    0

In [9]:
data_set=data_set.dropna(axis=0)

In [10]:
a=pd.DataFrame(data_set.isnull().sum())

In [11]:
a['# of null values']=a[0]
b = a[['# of null values']]


In [12]:
print ( "After Dropping null values: ")
print ( " # of rows, columns :", data_set.shape)

After Dropping null values: 
 # of rows, columns : (20433, 10)


In [13]:
ICD.display( b)

# of null values
longitude                          0
latitude                           0
housing_median_age                 0
total_rooms                        0
total_bedrooms                     0
population                         0
households                         0
median_income                      0
median_house_value                 0
ocean_proximity                    0

In [14]:
data_set['num_rooms']=data_set['total_rooms']/data_set['households']
data_set['num_bedrooms']=data_set['total_bedrooms']/data_set['households']
data_set['persons_per_house']=data_set['population']/data_set['households']
data_set.drop(['total_rooms','total_bedrooms','population','households'],axis=1,inplace=True)

In [15]:
data_set.head(10)

longitude  latitude  housing_median_age  median_income  median_house_value  \
0    -122.23     37.88                41.0         8.3252            452600.0   
1    -122.22     37.86                21.0         8.3014            358500.0   
2    -122.24     37.85                52.0         7.2574            352100.0   
3    -122.25     37.85                52.0         5.6431            341300.0   
4    -122.25     37.85                52.0         3.8462            342200.0   
5    -122.25     37.85                52.0         4.0368            269700.0   
6    -122.25     37.84                52.0         3.6591            299200.0   
7    -122.25     37.84                52.0         3.1200            241400.0   
8    -122.26     37.84                42.0         2.0804            226700.0   
9    -122.25     37.84                52.0         3.6912            261100.0   

  ocean_proximity  num_rooms  num_bedrooms  persons_per_house  
0        NEAR BAY   6.984127      1.023810           2.555556  
1        NEAR BAY   6.238137      0.971880           2.109842  
2        NEAR BAY   8.288136      1.073446           2.802260  
3        NEAR BAY   5.817352      1.073059           2.547945  
4        NEAR BAY   6.281853      1.081081           2.181467  
5        NEAR BAY   4.761658      1.103627           2.139896  
6        NEAR BAY   4.931907      0.951362           2.128405  
7        NEAR BAY   4.797527      1.061824           1.788253  
8        NEAR BAY   4.294118      1.117647           2.026891  
9        NEAR BAY   4.970588      0.990196           2.172269

In [16]:
featcols = {
  colname : tf.feature_column.numeric_column(colname) \
    for colname in 'housing_median_age,median_income,num_rooms,num_bedrooms,persons_per_house'.split(',')
}

In [17]:
featcols['longitude'] = tf.feature_column.bucketized_column(tf.feature_column.numeric_column('longitude'),
                                                   np.linspace(-124.3, -114.3, 5).tolist())
featcols['latitude'] = tf.feature_column.bucketized_column(tf.feature_column.numeric_column('latitude'),
                                                  np.linspace(32.5, 42, 10).tolist())


In [18]:
print((len(data_set)))

20433


In [19]:
mask=np.random.rand(len(data_set))<0.8

In [20]:
print( mask)

[ True  True  True ... False  True  True]


In [21]:
Training_data_set=data_set[mask]

In [22]:
Eval_data_set=data_set[~mask]

In [96]:
# Learning rate 0.3, 0.1, 0.03, 0.01
# batch_size 20-100
# num_epochs 1 - 50
scale = 100000
batch_size=30
num_epochs=50
learning_rate=0.03
NSteps=30000

In [97]:
train_input_fn = tf.estimator.inputs.pandas_input_fn(x = Training_data_set[list(featcols.keys())],
                                                    y = Training_data_set["median_house_value"] / scale,
                                                    num_epochs = num_epochs,
                                                    batch_size = batch_size,
                                                    shuffle = True)

In [98]:
Eval_input_fn=tf.estimator.inputs.pandas_input_fn(x=Eval_data_set[list(featcols.keys())],
                                                  y=Eval_data_set["median_house_value"]/scale,
                                                  num_epochs=num_epochs,
                                                  batch_size=len(Eval_data_set),
                                                  shuffle=False
)

In [99]:
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)


In [100]:
# outdir='./foldere'

In [101]:
model = tf.estimator.LinearRegressor(model_dir = outdir, feature_columns = featcols.values(), optimizer = optimizer )



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './foldere', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000FE244B1B70>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [102]:

model.train(input_fn=train_input_fn, steps=NSteps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./foldere\model.ckpt-23202
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 23202 into ./foldere\model.ckpt.
INFO:tensorflow:loss = 47.7514, step = 23203
INFO:tensorflow:global_step/sec: 251.088
INFO:tensorflow:loss = 5.3756833, step = 23303 (0.400 sec)
INFO:tensorflow:global_step/sec: 458.414
INFO:tensorflow:loss = 15.053745, step = 23403 (0.219 sec)
INFO:tensorflow:global_step/sec: 480.446
INFO:tensorflow:loss = 27.64035, step = 23503 (0.208 sec)
INFO:tensorflow:global_step/sec: 475.875
INFO:tensorflow:loss = 25.871689, step = 23603 (0.209 sec)
INFO:tensorflow:global_step/sec: 450.15
INFO:tensorflow:loss = 6.8576803, step = 23703 (0.222 sec)
INFO:tensorflow:global_step/sec: 456.317
INFO:tensorflow:loss = 27.053064, step

INFO:tensorflow:loss = 8.120415, step = 31103 (0.287 sec)
INFO:tensorflow:global_step/sec: 322.365
INFO:tensorflow:loss = 33.259857, step = 31203 (0.312 sec)
INFO:tensorflow:global_step/sec: 360.77
INFO:tensorflow:loss = 15.548943, step = 31303 (0.273 sec)
INFO:tensorflow:global_step/sec: 334.226
INFO:tensorflow:loss = 35.263832, step = 31403 (0.302 sec)
INFO:tensorflow:global_step/sec: 330.905
INFO:tensorflow:loss = 13.760766, step = 31503 (0.302 sec)
INFO:tensorflow:global_step/sec: 370.126
INFO:tensorflow:loss = 12.62391, step = 31603 (0.268 sec)
INFO:tensorflow:global_step/sec: 350.642
INFO:tensorflow:loss = 83.678345, step = 31703 (0.286 sec)
INFO:tensorflow:global_step/sec: 364.721
INFO:tensorflow:loss = 14.561314, step = 31803 (0.276 sec)
INFO:tensorflow:global_step/sec: 361.412
INFO:tensorflow:loss = 6.9887357, step = 31903 (0.275 sec)
INFO:tensorflow:global_step/sec: 360.77
INFO:tensorflow:loss = 16.11322, step = 32003 (0.277 sec)
INFO:tensorflow:global_step/sec: 358.183
INFO:

INFO:tensorflow:global_step/sec: 342.237
INFO:tensorflow:loss = 25.049557, step = 39403 (0.291 sec)
INFO:tensorflow:global_step/sec: 355.633
INFO:tensorflow:loss = 11.654161, step = 39503 (0.282 sec)
INFO:tensorflow:global_step/sec: 317.249
INFO:tensorflow:loss = 34.50335, step = 39603 (0.315 sec)
INFO:tensorflow:global_step/sec: 362.077
INFO:tensorflow:loss = 24.707647, step = 39703 (0.276 sec)
INFO:tensorflow:global_step/sec: 358.184
INFO:tensorflow:loss = 11.846287, step = 39803 (0.281 sec)
INFO:tensorflow:global_step/sec: 363.394
INFO:tensorflow:loss = 29.683262, step = 39903 (0.272 sec)
INFO:tensorflow:global_step/sec: 363.394
INFO:tensorflow:loss = 35.891846, step = 40003 (0.273 sec)
INFO:tensorflow:global_step/sec: 363.396
INFO:tensorflow:loss = 14.669132, step = 40103 (0.277 sec)
INFO:tensorflow:global_step/sec: 359.47
INFO:tensorflow:loss = 21.225206, step = 40203 (0.279 sec)
INFO:tensorflow:global_step/sec: 359.473
INFO:tensorflow:loss = 19.521816, step = 40303 (0.277 sec)
IN

INFO:tensorflow:loss = 20.566036, step = 47603 (0.279 sec)
INFO:tensorflow:global_step/sec: 356.905
INFO:tensorflow:loss = 9.398984, step = 47703 (0.279 sec)
INFO:tensorflow:global_step/sec: 363.394
INFO:tensorflow:loss = 70.182816, step = 47803 (0.274 sec)
INFO:tensorflow:global_step/sec: 362.075
INFO:tensorflow:loss = 27.416718, step = 47903 (0.279 sec)
INFO:tensorflow:global_step/sec: 356.909
INFO:tensorflow:loss = 6.6226306, step = 48003 (0.280 sec)
INFO:tensorflow:global_step/sec: 363.392
INFO:tensorflow:loss = 3007.2456, step = 48103 (0.271 sec)
INFO:tensorflow:global_step/sec: 364.72
INFO:tensorflow:loss = 10.13191, step = 48203 (0.275 sec)
INFO:tensorflow:global_step/sec: 358.184
INFO:tensorflow:loss = 7.1229095, step = 48303 (0.281 sec)
INFO:tensorflow:global_step/sec: 363.394
INFO:tensorflow:loss = 11.593556, step = 48403 (0.273 sec)
INFO:tensorflow:global_step/sec: 358.186
INFO:tensorflow:loss = 14.24515, step = 48503 (0.283 sec)
INFO:tensorflow:global_step/sec: 359.471
INFO